# Assignment 5

###### Created by Qihang Ma -- 2023.03.21

In [1]:
import warnings
warnings.filterwarnings("ignore")
from risk_mgmt import VaR, calculation
import pandas as pd
import numpy as np
import datetime as dt
from scipy.stats import norm
from scipy.optimize import fsolve, minimize
import inspect
import statsmodels.api as sm

## Problem 1 - Calculate greeks with GBSM & Binary Tree

Assume you a call and a put option with the following:  

    ● Current Stock Price $165
    ● Strike Price $165
    ● Current Date 03/13/2022
    ● Options Expiration Date 04/15/2022
    ● Risk Free Rate of 4.25%
    ● Continuously Compounding Coupon of 0.53%
    
Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation. Compare the values between the two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:

    ● Pays dividend on 4/11/2022 of $0.88
    
Calculate the value of the call and the put. Calculate the Greeks of each. 

What is the sensitivity of the put and call to a change in the dividend amount?

### Implement Black-Scholes closed form

In [2]:
class black_scholes_matrix:
    def __init__(self, S0, K, T, r, q, sigma, option='call'):
        """
        Initializes the Black-Scholes model with the given parameters.
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.q = q
        self.sigma = sigma
        self.option = option
        
        self.d1 = (np.log(self.S0 / self.K) + (self.r - self.q + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        self.d2 = self.d1 - self.sigma * np.sqrt(self.T)
    
    def price(self):
        """
        Calculates the theoretical price of the option.
        """
        if self.option == 'call':
            price = self.S0 * np.exp(-self.q * self.T) * norm.cdf(self.d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2)
        elif self.option == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.S0 * np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return price
    
    def delta(self):
        """
        Calculates the Delta of the option.
        """
        if self.option == 'call':
            delta = np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option == 'put':
            delta = -np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return delta
    
    def gamma(self):
        """
        Calculates the Gamma of the option.
        """
        gamma = np.exp(-self.q * self.T) * norm.pdf(self.d1) / (self.S0 * self.sigma * np.sqrt(self.T))
        
        return gamma
    
    def vega(self):
        """
        Calculates the Vega of the option.
        """
        vega = self.S0 * np.exp(-self.q * self.T) * norm.pdf(self.d1) * np.sqrt(self.T)
        
        return vega
    
    def theta(self):
        """
        Calculates the Theta of the option.
        """
        if self.option == 'call':
            theta = -self.S0 * np.exp(-self.q * self.T) * norm.pdf(self.d1) * self.sigma / (2 * np.sqrt(self.T)) - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2) + self.q * self.S0 * np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option == 'put':
            theta = -self.S0 * np.exp(-self.q * self.T) * norm.pdf(self.d1) * self.sigma / (2 * np.sqrt(self.T)) + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.q * self.S0 * np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return theta
    
    def rho(self):
        """
        Calculates the Rho of the option.
        """
        
        if self.option == 'call':
            rho = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(self.d2)
        elif self.option == 'put':
            rho = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-self.d2)

        return rho


    def carry_rho(self):
        """
        Calculates the Carry Rho of the option.
        """
        if self.option == 'call':
            carry_rho =  self.S0 * self.T * np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option == 'put':
            carry_rho =  - self.S0 * self.T * np.exp(-self.q * self.T) * norm.cdf(-self.d1)

        return carry_rho

    
    def greeks(self):
        """
        Calculates and returns all the greeks of the option as a dictionary.
        """
        delta = self.delta()
        gamma = self.gamma()
        vega = self.vega()
        theta = self.theta()
        rho = self.rho()
        carry_rho = self.carry_rho()
        
        return {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho, "Carry Rho": carry_rho}


In [3]:
def black_scholes(S0, K, T, r, q, sigma, option='call'):
    """
    Calculates the theoretical price of a European-style call or put option on a stock, using the Black-Scholes model.
    
    Parameters:
    - S0: the current stock price
    - K: the strike price of the option
    - T: the time to maturity of the option, expressed in years
    - r: the risk-free interest rate, expressed as a decimal
    - q: the continuously compounding coupon yield of the stock, expressed as a decimal
    - sigma: the implied volatility of the stock, expressed as a decimal
    - option: a string that indicates whether the option is a call or put option, default 'call'
    
    Returns:
    - price: the theoretical price of the option
    """
    
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option == 'call':
        price = S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)
    
    return price

### Implement the binomial tree valuation for American options with and without discrete dividends

In [4]:
def binomial_tree_american_continous(S0, K, T, r, q, sigma, N=200, option_type='call'):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp((r-q)*dt)-d)/(u-d)
    pd = 1-pu
    df = np.exp(-r*dt)
    z = 1 if option_type == 'call' else -1
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes, dtype = float)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i,j)
            price = S0*u**i*d**(j-i)
            optionValues[idx] = max(0,z*(price-K))
            if j < N:
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
    return optionValues[0]

In [5]:
def binomial_tree_american_discrete(S0, K, r, T, sigma, N, option_type, dividend_dates=None, dividend_amounts=None):
    if dividend_dates is None or dividend_amounts is None or (len(dividend_amounts)==0) or (len(dividend_dates)==0):
        return binomial_tree_american_continous(S0, K, T, r, 0, sigma, N, option_type)
    elif dividend_dates[0] > N:
        return binomial_tree_american_continous(S0, K, T, r, 0, sigma, N, option_type)

    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(r*dt)-d)/(u-d)
    pd = 1-pu
    df = np.exp(-r*dt)
    z = 1 if option_type == 'call' else -1
    
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
   
    nDiv = len(dividend_dates)
    nNodes = nNodeFunc(dividend_dates[0])

    optionValues = np.empty(nNodes, dtype = float)

    for j in range(dividend_dates[0],-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = S0*u**i*d**(j-i)       
            
            if j < dividend_dates[0]:
                #times before the dividend working backward induction
                optionValues[idx] = max(0,z*(price-K))
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
                
            else:
                no_ex= binomial_tree_american_discrete(price-dividend_amounts[0], K, r, T-dividend_dates[0]*dt, sigma, N-dividend_dates[0], option_type, [x- dividend_dates[0] for x in dividend_dates[1:nDiv]], dividend_amounts[1:nDiv] )
                ex =  max(0,z*(price-K))
                optionValues[idx] = max(no_ex,ex)

    return optionValues[0]

### Implement a finite difference derivative calculation

In [6]:
# calculate first order derivative
def first_order_der(func, x, delta):
    return (func(x * (1 + delta)) - func(x * (1 - delta))) / (2 * x * delta)

# calculate second order derivative
def second_order_der(func, x, delta):
    return (func(x * (1 + delta)) + func(x * (1 - delta)) - 2 * func(x)) / (x * delta) ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-5):
  # initialize for argument names and order
    arg_names = list(inspect.signature(func).parameters.keys())
    derivative_fs = {1: first_order_der, 2: second_order_der}

    def partial_derivative(*args, **kwargs):
        # parse argument names and order
        args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
        arg_val = args_dict.pop(arg_name)

        def partial_f(x):
            p_kwargs = {arg_name:x, **args_dict}
            return func(**p_kwargs)
        return derivative_fs[order](partial_f, arg_val, delta)
    return partial_derivative

In [7]:
class OptionGreekCalculator:
    def __init__(self, option_price_func, S, K, r, T, sigma, option_type, q = None, N = None, dividend_dates=None, dividend_amounts=None):
        self.option_price_func = option_price_func
        self.S = S
        self.K = K
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.N = N
        self.option_type = option_type
        self.dividend_dates = dividend_dates
        self.dividend_amounts = dividend_amounts
        
    def __call__(self, *args, **kwargs):
        return self.option_price_func(*args, **kwargs)
    
    def delta(self):
        delta_calculator = cal_partial_derivative(self.option_price_func, 1, 'S0')
        if self.option_price_func == black_scholes:
            delta = delta_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == binomial_tree_american_continous:
            delta = delta_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == binomial_tree_american_discrete:
            delta = delta_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return delta
    
    def gamma(self):
        gamma_calculator = cal_partial_derivative(self.option_price_func, 2, 'S0')
        if self.option_price_func == black_scholes:
            gamma = gamma_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == binomial_tree_american_continous:
            gamma = gamma_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == binomial_tree_american_discrete:
            gamma = gamma_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return gamma
    
    def vega(self):
        vega_calculator = cal_partial_derivative(self.option_price_func, 1, 'sigma')
        if self.option_price_func == black_scholes:
            vega = vega_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == binomial_tree_american_continous:
            vega = vega_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == binomial_tree_american_discrete:
            vega = vega_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return vega
    
    def theta(self):
        theta_calculator = cal_partial_derivative(self.option_price_func, 1, 'T')
        if self.option_price_func == black_scholes:
            theta = theta_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == binomial_tree_american_continous:
            theta = theta_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == binomial_tree_american_discrete:
            theta = theta_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return -theta
    
    def rho(self):
        rho_calculator = cal_partial_derivative(self.option_price_func, 1, 'r')
        if self.option_price_func == black_scholes:
            rho = rho_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == binomial_tree_american_continous:
            rho = rho_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == binomial_tree_american_discrete:
            rho = rho_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return rho 
    
    def carry_rho(self):
        carry_rho_calculator = cal_partial_derivative(self.option_price_func, 1, 'q')
        if self.option_price_func == black_scholes:
            carry_rho = self.rho() - carry_rho_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == binomial_tree_american_continous:
            carry_rho = self.rho() - carry_rho_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == binomial_tree_american_discrete:
            return
        return carry_rho 
    
    def sensitivity_to_dividend(self):
        if self.dividend_dates is None or self.dividend_amounts is None or self.option_price_func != binomial_tree_american_discrete:
            return
        delta = 1e-3
        div_amounts_1 = [self.dividend_amounts[0]+delta] + self.dividend_amounts[1:]
        div_amounts_2 = [self.dividend_amounts[0]-delta] + self.dividend_amounts[1:]
        V1 = binomial_tree_american_discrete(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, div_amounts_1)    
        V2 = binomial_tree_american_discrete(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, div_amounts_2)    
        sensitivity_to_dividend = (V1 - V2) / (2*delta)
        return sensitivity_to_dividend
    
    def greeks(self):
        delta = self.delta()
        gamma = self.gamma()
        vega = self.vega()
        theta = self.theta()
        rho = self.rho()
        carry_rho = self.carry_rho() 
        sensitivity_to_dividend = self.sensitivity_to_dividend()
        
        return {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho, "Carry Rho": carry_rho, "Senstivity to Dividend": sensitivity_to_dividend}
        

### Calculate Greeks

In [8]:
S0 = 165
K = 165
T = (dt.datetime(2023,4,15) - dt.datetime(2023,3,13)).days / 365
r = 0.0425
q = 0.0053
sigma = 0.2
N = 200
dividend_dates = [round((dt.datetime(2023,4,11)-dt.datetime(2023,3,13)).days/(dt.datetime(2023,4,15)-dt.datetime(2023,3,13)).days*N)]
dividend_amounts = [0.88]


In [9]:
american_call_value = binomial_tree_american_discrete(S0, K, r, T, sigma, N, "call", dividend_dates, dividend_amounts)
american_call_value

4.120022561302723

In [10]:
american_put_value = binomial_tree_american_discrete(S0, K, r, T, sigma, N, "put", dividend_dates, dividend_amounts)
american_put_value

4.109950677609662

In [11]:
gbsm_call_derivative = OptionGreekCalculator(black_scholes, S0, K, r, T, sigma, 'call', q)
gbsm_call_derivative_greeks = gbsm_call_derivative.greeks()
greeks = pd.DataFrame.from_dict(gbsm_call_derivative_greeks, orient='index', columns=['gbsm_call_derivative'])

gbsm_put_derivative = OptionGreekCalculator(black_scholes, S0, K, r, T, sigma, 'put', q)
gbsm_put_derivative_greeks = gbsm_put_derivative.greeks()
greeks['gbsm_put_derivative'] = pd.DataFrame.from_dict(gbsm_put_derivative_greeks, orient='index')

gbsm_call_closed = black_scholes_matrix(S0, K, T, r, q, sigma, "call")
gbsm_call_closed_greeks = gbsm_call_closed.greeks()
greeks['gbsm_call_closed'] = pd.DataFrame.from_dict(gbsm_call_closed_greeks, orient='index')

gbsm_put_closed = black_scholes_matrix(S0, K, T, r, q, sigma, "put")
gbsm_put_closed_greeks = gbsm_put_closed.greeks()
greeks['gbsm_put_closed'] = pd.DataFrame.from_dict(gbsm_put_closed_greeks, orient='index')

bt_american_call = OptionGreekCalculator(binomial_tree_american_discrete, S0, K, r, T, sigma, 'call', 0, N, dividend_dates, dividend_amounts)
bt_american_call_greeks = bt_american_call.greeks()
greeks['bt_american_call'] = pd.DataFrame.from_dict(bt_american_call_greeks, orient='index')

bt_american_put = OptionGreekCalculator(binomial_tree_american_discrete, S0, K, r, T, sigma, 'put', 0, N, dividend_dates, dividend_amounts)
bt_american_put_greeks = bt_american_put.greeks()
greeks['bt_american_put'] = pd.DataFrame.from_dict(bt_american_put_greeks, orient='index')

greeks

,gbsm_call_derivative,gbsm_put_derivative,gbsm_call_closed,gbsm_put_closed,bt_american_call,bt_american_put
Delta,0.471430,-0.524735,0.534009,-0.465512,5.385793e-01,-4.930986e-01
Gamma,0.058285,0.058285,0.040038,0.040038,5.872254e-09,-3.914836e-09
Vega,13.487815,13.487815,19.710180,19.710180,1.951501e+01,1.982413e+01
Theta,-25.102764,-39.089058,-24.898522,-18.786997,-2.479532e+01,-1.853281e+01
Rho,3.203014,-3.807907,7.583586,-7.277011,6.829826e+00,-7.219850e+00
Carry Rho,6.508918,-7.487609,7.966246,-6.944416,NaN,NaN
Senstivity to Dividend,NaN,NaN,NaN,NaN,-9.355484e-02,5.125798e-01


## Problem 2 - Calculate the Value of the Options for AAPL

Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :

    ● American Options
    ● Current Date 03/03/2023
    ● Current AAPL price is 165
    ● Risk Free Rate of 4.25%
    ● Dividend Payment of $1.00 on 3/15/2023
    
Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). 

Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a $ loss, not percentages. Compare these results to last week’s results.

### Load the data

In [12]:
aapl_option = pd.read_csv('problem2.csv')
aapl_option['ExpirationDate'] = pd.to_datetime(aapl_option['ExpirationDate'])
aapl_option

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80
5,CallSpread,Option,AAPL,-1,Call,2023-04-21,160.0,2.21
6,PutSpread,Option,AAPL,1,Put,2023-04-21,150.0,4.85
7,PutSpread,Option,AAPL,-1,Put,2023-04-21,140.0,1.84
8,Stock,Stock,AAPL,1,NaN,NaT,NaN,151.03
9,Call,Option,AAPL,1,Call,2023-04-21,150.0,6.80


In [13]:
price = pd.read_csv('DailyPrices.csv')
aapl_return = pd.DataFrame(calculation.return_calculate(price, 'log')["AAPL"])
aapl_norm = aapl_return - aapl_return.mean()

### Calculate the implied volatility with binary tree model

In [14]:
def implied_volatility_bt(S0, K, r, T, price, N, option, dividend_dates=None, dividend_amounts=None):
    f1 = lambda z: (binomial_tree_american_discrete(S0, K, r, T, z, N, option, dividend_dates, dividend_amounts)-price)
    return fsolve(f1, x0 = 0.2)[0]

### Calculate portfolio values

In [15]:
def calculate_portfolio_values(portfolios, underlying_value, days_ahead=0):
    portfolio_values = pd.DataFrame(index=portfolios["Portfolio"].unique(), columns=[underlying_value])
    portfolio_values = portfolio_values.fillna(0)
    
    for i, portfolio in portfolios.iterrows():

        if portfolio["Type"] == "Stock":
            asset_value = underlying_value
            
        else:
            K = portfolio["Strike"]
            T = ((portfolio["ExpirationDate"] - current_date).days - days_ahead) / 365
            price = portfolio["CurrentPrice"]
            dividend_dates = [round(((dt.datetime(2023,3,15) - current_date).days - days_ahead) / ((portfolio["ExpirationDate"] - current_date).days - days_ahead) * N)]
            dividend_amounts = [1]
            sigma = portfolio["ImpliedVol"]
            
            asset_values = []
            for underlying_prices in np.atleast_1d(underlying_value):
                option_values = (binomial_tree_american_discrete(underlying_prices, K, r, T, sigma, 50, portfolio.loc['OptionType'].lower(), dividend_dates, dividend_amounts))
                asset_values.append(option_values)
            asset_value = np.array(asset_values)
        
        portfolio_values.loc[portfolio["Portfolio"], :] += portfolio["Holding"] * asset_value
        
    return portfolio_values

### Simulate the price and calculate the values

In [16]:
def simulate_prices(daily_returns, current_price, days=1, n_simulation = 1000):

    mu, std = norm.fit(daily_returns)
    simulated_returns = np.random.normal(mu, std, (days, n_simulation))
    simulate_prices = current_price * np.exp(simulated_returns.cumsum(axis=0))
    
    return simulate_prices

In [17]:
current_date = dt.datetime(2023,3,3)
S0 = 165
r = 0.0425
days_ahead = 0

implied_vols = []

for i, portfolio in aapl_option.iterrows():

    if portfolio["Type"] == "Stock":
        implied_vols.append(None)

    else:
        K = portfolio["Strike"]
        T = ((portfolio["ExpirationDate"] - current_date).days - days_ahead) / 365
        price = portfolio["CurrentPrice"]
        dividend_dates = [round(((dt.datetime(2023,3,15) - current_date).days - days_ahead) / ((portfolio["ExpirationDate"] - current_date).days - days_ahead) * N)]
        dividend_amounts = [1]
        sigma = implied_volatility_bt(S0, K, r, T,  price, 50, portfolio.loc['OptionType'].lower(), dividend_dates, dividend_amounts)
        implied_vols.append(sigma)

aapl_option["ImpliedVol"] = implied_vols

current_values = calculate_portfolio_values(aapl_option, S0, 0)

In [18]:
np.random.seed(123)
underlying_prices = pd.DataFrame(simulate_prices(aapl_norm, S0, 10))
simulate_portfolio_values = calculate_portfolio_values(aapl_option, underlying_prices.loc[9:].values[0], 10)

In [19]:
merged_df = pd.merge(simulate_portfolio_values, current_values, left_index=True, right_index=True)
price_change = merged_df.sub(merged_df[S0], axis=0).drop(S0, axis=1)
price_change.columns = price_change.columns.str[0]

portfolio_metrics = pd.DataFrame(index=aapl_option["Portfolio"].unique(), columns=["Mean", "VaR", "ES"])
portfolio_metrics = portfolio_metrics.fillna(0)

for index, row in price_change.iterrows():
    mean = row.values.mean()
    var = VaR.calculate_var(row.values)
    es = VaR.calculate_es(row.values, var)
    
    portfolio_metrics.loc[index, "Mean"] = mean
    portfolio_metrics.loc[index, "VaR"] = var
    portfolio_metrics.loc[index, "ES"] = es
    
portfolio_metrics

,Mean,VaR,ES
Straddle,0.154010,10.517501,10.946795
SynLong,0.744245,21.293018,24.214408
CallSpread,-1.812549,9.944242,9.944242
PutSpread,-0.191446,2.290312,2.498623
Stock,0.774051,17.050425,22.095201
Call,0.449127,15.836365,15.836365
Put,-0.295117,3.967262,4.234719
CoveredCall,-0.271751,6.186181,11.230957
ProtectedPut,0.601873,12.910778,15.454527


### Calculate with Delta-Normal

In [20]:
delta_calculator =  cal_partial_derivative(binomial_tree_american_discrete, 1, 'S0')

for i in range(len(aapl_option)):
    
    K = aapl_option.loc[i,"Strike"]
    price = aapl_option.loc[i,"CurrentPrice"]
    T = ((aapl_option.loc[i, "ExpirationDate"] - current_date).days - 10) / 365
    sigma = aapl_option.loc[i,"ImpliedVol"]
    option_type = aapl_option.loc[i,"OptionType"]
    dividend_dates = [round(((dt.datetime(2023,3,15) - current_date).days - 10) / ((portfolio["ExpirationDate"] - current_date).days - 10) * N)]
    dividend_amounts = [1]
    
    if aapl_option.loc[i,"Type"] != "Stock":
        aapl_option.loc[i, "Delta"] = delta_calculator(S0, K, r, T, sigma, 50, option_type, dividend_dates, dividend_amounts) * aapl_option.loc[i,'Holding']
    else:
        aapl_option.loc[i, "Delta"] = 1 * aapl_option.loc[i,'Holding']

In [21]:
delta = aapl_option.groupby("Portfolio")['Delta'].sum().apply(lambda x: -x * (underlying_prices.loc[9:].values[0]-165))
delta_df = pd.DataFrame(np.array([delta.values[i].reshape(-1) for i in range(len(delta))]), index=delta.keys(), columns=underlying_prices.loc[9:].values[0])

price_change_delta = price_change.add(delta_df)

portfolio_metrics_1 = pd.DataFrame(index=aapl_option["Portfolio"].unique(), columns=["Mean", "VaR", "ES"])
portfolio_metrics_1 = portfolio_metrics_1.fillna(0)

for index, row in price_change_delta.iterrows():
    mean = row.values.mean()
    var = VaR.calculate_var(row.values)
    es = VaR.calculate_es(row.values, var)
    
    portfolio_metrics_1.loc[index, "Mean"] = mean
    portfolio_metrics_1.loc[index, "VaR"] = var
    portfolio_metrics_1.loc[index, "ES"] = es
    
portfolio_metrics_1


,Mean,VaR,ES
Straddle,0.317031,13.727628,14.167020
SynLong,0.581224,17.702074,19.560997
CallSpread,-1.812549,9.944242,9.944242
PutSpread,-0.105243,0.438345,0.446340
Stock,0.000000,0.000000,-0.000000
Call,0.449127,15.836365,15.836365
Put,-0.132097,0.963082,0.963664
CoveredCall,-1.045802,20.401018,25.699641
ProtectedPut,-0.062989,0.779199,0.780872


## Problem 3 - Fama, French

Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

    AAPL FB UNH MA MSFT
    NVDA HD PFE AMZN BRK-B
    PG XOM TSLA JPM V
    DIS GOOGL JNJ BAC CSCO
    
Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100) to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock. Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0425. Find the super efficient portfolio

### Load data

In [22]:
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})

factor = (ff.join(mom, how='right') / 100).loc['2013-1-31':]

In [23]:
prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date'])
all_returns = pd.DataFrame(calculation.return_calculate(prices)).set_index('Date')

In [24]:
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
dataset = all_returns[stocks].join(factor)

subset = dataset.dropna()

### Find Alpha and Beta

In [25]:
X = subset[factors]
X = sm.add_constant(X)

y = subset[stocks].sub(subset['RF'], axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])


for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

### Calculate the Expected Return

In [26]:
sub_return = pd.DataFrame(np.dot(factor[factors],betas.T), index=factor.index, columns=betas.index)
merge_return = pd.merge(sub_return,factor['RF'], left_index=True, right_index=True)
daily_expected_returns = merge_return.add(merge_return['RF'],axis=0).drop('RF',axis=1).add(alphas.T.loc['Alpha'], axis=1)

expected_annual_return = ((daily_expected_returns+1).cumprod().tail(1) ** (1/daily_expected_returns.shape[0]) - 1) * 252

expected_annual_return

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.157144,0.017941,0.2538,0.222901,0.155944,0.279721,0.120591,0.076962,-0.042945,0.129923,0.08154,0.521821,-0.033253,0.098273,0.241054,-0.155372,-0.017075,0.124206,-0.112301,0.147807


In [27]:
covariance_matrix = dataset[stocks].cov() * 252
covariance_matrix

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.126877,0.139557,0.037447,0.081272,0.102937,0.171265,0.066193,0.032745,0.122117,0.055520,0.036945,0.037700,0.154880,0.058646,0.071406,0.087399,0.111906,0.022740,0.066245,0.066123
META,0.139557,0.400843,0.017102,0.102465,0.142255,0.240599,0.098845,0.045091,0.194794,0.061619,0.033637,0.020759,0.173121,0.073973,0.085022,0.127047,0.182074,0.021329,0.088791,0.076719
UNH,0.037447,0.017102,0.060922,0.031117,0.036318,0.046531,0.026045,0.032068,0.034737,0.028173,0.027861,0.026843,0.039128,0.033321,0.029590,0.022467,0.029806,0.022981,0.034744,0.028847
MA,0.081272,0.102465,0.031117,0.095762,0.079856,0.137369,0.056792,0.033440,0.096194,0.047520,0.031163,0.030837,0.097521,0.058309,0.082400,0.076987,0.079016,0.017382,0.063399,0.051734
MSFT,0.102937,0.142255,0.036318,0.079856,0.127839,0.175956,0.070916,0.035082,0.133856,0.052676,0.033859,0.031304,0.131911,0.056714,0.068230,0.088227,0.120259,0.020323,0.065226,0.060804
NVDA,0.171265,0.240599,0.046531,0.137369,0.175956,0.403814,0.112055,0.046362,0.221364,0.084290,0.041944,0.054455,0.291392,0.098399,0.118145,0.157837,0.188012,0.021758,0.113397,0.098663
HD,0.066193,0.098845,0.026045,0.056792,0.070916,0.112055,0.097074,0.033271,0.096833,0.042310,0.034414,0.015936,0.077754,0.043555,0.050164,0.063936,0.069626,0.022364,0.046487,0.048412
PFE,0.032745,0.045091,0.032068,0.033440,0.035082,0.046362,0.033271,0.070517,0.037274,0.031202,0.027470,0.019979,0.022283,0.031862,0.030650,0.024522,0.029355,0.027603,0.030812,0.029531
AMZN,0.122117,0.194794,0.034737,0.096194,0.133856,0.221364,0.096833,0.037274,0.242775,0.065772,0.030153,0.037446,0.187717,0.070819,0.083373,0.124253,0.149924,0.022616,0.085241,0.071419
BRK-B,0.055520,0.061619,0.028173,0.047520,0.052676,0.084290,0.042310,0.031202,0.065772,0.050175,0.024839,0.034026,0.061541,0.047076,0.042080,0.051383,0.056281,0.019263,0.050820,0.040421


### Find the super efficient portfolio

In [28]:
def super_efficient_portfolio(returns, rf_rate, cov_matrix):
    if len(returns.shape) == 1:
        num_assets = returns.shape[0]
    else:
        num_assets = returns.shape[1]
    
    # Define objective function to minimize
    def neg_sharpe_ratio(weights):
        port_return = np.sum(returns * weights)
        port_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (port_return - rf_rate) / port_std_dev
        return -sharpe_ratio
    
    # Define constraints: weights sum to 1, all weights are non-negative
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    
    # Define bounds: weights are between 0 and 1
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Solve for optimal weights
    init_weights = np.ones(num_assets) / num_assets  # start with equal weights
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Return optimal weights and Sharpe ratio of resulting portfolio
    opt_weights = opt_result.x
    opt_port_return = np.sum(returns * opt_weights)
    opt_port_std_dev = np.sqrt(np.dot(opt_weights.T, np.dot(cov_matrix, opt_weights)))
    opt_sharpe_ratio = (opt_port_return - rf_rate) / opt_port_std_dev
    return opt_weights*100, opt_sharpe_ratio


In [29]:
weights, sharpe_ratio = super_efficient_portfolio(expected_annual_return.values[0], 0.0425, covariance_matrix)

print("The Portfolio's Sharpe Ratio is: {:.2f}" .format(sharpe_ratio))

weights = pd.DataFrame(weights, index=expected_annual_return.columns, columns=['weight %']).round(2).T
weights

The Portfolio's Sharpe Ratio is: 1.47


,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
weight %,0.0,0.0,22.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.44,0.0,0.0,12.93,0.0,0.0,7.05,0.0,0.0
